# Definitions

In [ ]:
from matplotlib import pyplot as plt
import dask.array as da
import numpy as np
import xarray as xr

In [ ]:
from kaleidoscope.val.plots import HistogramPlot
from kaleidoscope.val.plots import WorldPlot

In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
def plot_histogram(data, title: str):
    fig = HistogramPlot().plot(
        data=data,
        xlabel="normalized error",
        ylabel="probability density",
        fn=f"hist-res-{title}",
        title=title,
        plot_size=(4.8, 3.6),
        bins=11,
        hist_range=(-5.5, 5.5),
        density=True,
    )
    fig.clear()

In [ ]:
def plot_histogram_mean(data, dims: str | list[str], title: str):
    fig = HistogramPlot().plot(
        data=data.mean(dims),
        xlabel="mean normalized error",
        ylabel="probability density",
        fn=f"hist-mean-{title}",
        title=title,
        plot_size=(4.8, 3.6),
        bins=11,
        hist_range=(-1.1, 1.1),
        density=True,
    )
    fig.clear()

In [ ]:
def plot_histogram_std(data, dims: str | list[str], title: str):
    fig = HistogramPlot().plot(
        data=da.sqrt(da.square(data).mean(dims)),
        xlabel="std. dev. normalized errors",
        ylabel="probability density",
        fn=f"hist-std-{title}",
        title=title,
        plot_size=(4.8, 3.6),
        bins=9,
        hist_range=(0.1, 1.9),
        density=True,
    )
    fig.clear()

In [ ]:
def plot_histogram_range(data, dims: str | list[str], title: str):
    fig = HistogramPlot().plot(
        data=data.max(dims) - data.min(dims),
        xlabel="range of normalized errors",
        ylabel="probability density",
        fn=f"hist-range-{title}",
        title=title,
        plot_size=(4.8, 3.6),
        bins=15,
        hist_range=(0.0, 6.0),
        density=True,
    )
    fig.clear()

In [ ]:
def plot_mean(
    data,
    dims: str | list[str],
    title: str,
    vmin=-0.5,
    vmax=0.5,
    x: str | None = None,
    y: str | None = None,
):
    fig = WorldPlot().plot(
        data=data.mean(dims),
        title=title,
        fn=f"map-mean-{title}",
        plot_size=(9.6, 3.6),
        cbar_label="mean normalized error",
        vmin=vmin,
        vmax=vmax,
        x=x,
        y=y,
    )
    fig.clear()

In [ ]:
def plot_std(
    data,
    dims: str | list[str],
    title: str,
    vmin=0.5,
    vmax=1.5,
    x: str | None = None,
    y: str | None = None,
):
    fig = WorldPlot().plot(
        data=da.sqrt(da.square(data).mean(dims)),
        title=title,
        fn=f"map-std-{title}",
        plot_size=(9.6, 3.6),
        cbar_label="std. dev. normalized errors",
        vmin=vmin,
        vmax=vmax,
        x=x,
        y=y,
    )
    fig.clear()

In [ ]:
def plot_range(
    data,
    dims: str | list[str],
    title: str,
    vmin=1.0,
    vmax=5.0,
    x: str | None = None,
    y: str | None = None,
):
    fig = WorldPlot().plot(
        data=(data.max(dim=dims) - data.min(dim=dims)),
        title=title,
        fn=f"map-range-{title}",
        plot_size=(9.6, 3.6),
        cbar_label="range of normalized errors",
        vmin=vmin,
        vmax=vmax,
        x=x,
        y=y,
    )
    fig.clear()

# ESA SCOPE carbonate system (D. J. Ford)

In [ ]:
ds = xr.open_mfdataset(
    "./Ford_et_al_UExP-FNN-U_physics_carbonatesystem_ESASCOPE_v5.*.nc",
    engine="h5netcdf",
    chunks={"time": 12},
    combine="nested",
    concat_dim="i",
).transpose("i", "time", "latitude", "longitude")

In [ ]:
ds

In [ ]:
config = {
    "fco2": {
        "name": "Fugacity of CO2 in sea water",
        "uncertainty": "fco2_tot_unc",
        "coverage": 2.0,
    },
    "flux": {
        "name": "Air-sea CO2 flux",
        "uncertainty": "flux_unc",
        "relative": True,
        "coverage": 2.0,
    },
    "ta": {
        "name": "Total alkalinity",
        "uncertainty": "ta_tot_unc",
        "coverage": 2.0,
    },
    "dic": {
        "name": "Dissolved inorganic carbon",
        "uncertainty": "dic_tot_unc",
        "coverage": 2.0,
    },
    "pH": {
        "name": "pH",
        "uncertainty": "pH_tot_unc",
        "coverage": 2.0,
    },
    "saturation_aragonite": {
        "name": "Saturation state of aragonite",
        "uncertainty": "saturation_aragonite_tot_unc",
        "coverage": 2.0,
    },
}

In [ ]:
def xu(ds, v, a):
    x = ds[v][0]
    u = ds[a["uncertainty"]][0] / a.get("coverage", 1.0)
    if a.get("relative", False):
        u = u * x
    return x, u

In [ ]:
for v, a in config.items():
    x, u = xu(ds, v, a)
    res = (ds[v][1:] - x) / u
    plot_histogram(
        res,
        title=a["name"],
    )
    plot_mean(
        res,
        dims=["i", "time"],
        title=a["name"],
    )
    plot_std(
        res,
        dims=["i", "time"],
        title=a["name"],
    )
    plot_range(
        res,
        dims=["i", "time"],
        title=a["name"],
    )

In [ ]:
ds.close()

# ESA CCI Ocean Colour

In [ ]:
ds = xr.open_mfdataset(
    "./ESACCI-OC-L3S-OC_PRODUCTS-MERGED-1M_MONTHLY_4km_GEO_PML_OCx_QAA-200001-fv6.0.*.nc",
    engine="h5netcdf",
    chunks={"lat": 1080, "lon": 1080},
    combine="nested",
    concat_dim="i",
)

In [ ]:
ds

In [ ]:
config = {
    "Rrs_412": {
        "name": "Remote sensing reflectance (412 nm)",
        "bias": "Rrs_412_bias",
        "rmsd": "Rrs_412_rmsd",
    },
    "Rrs_443": {
        "name": "Remote sensing reflectance (443 nm)",
        "bias": "Rrs_443_bias",
        "rmsd": "Rrs_443_rmsd",
    },
    "Rrs_490": {
        "name": "Remote sensing reflectance (490 nm)",
        "bias": "Rrs_490_bias",
        "rmsd": "Rrs_490_rmsd",
    },
    "Rrs_510": {
        "name": "Remote sensing reflectance (510 nm)",
        "bias": "Rrs_510_bias",
        "rmsd": "Rrs_510_rmsd",
    },
    "Rrs_560": {
        "name": "Remote sensing reflectance (560 nm)",
        "bias": "Rrs_560_bias",
        "rmsd": "Rrs_560_rmsd",
    },
    "Rrs_665": {
        "name": "Remote sensing reflectance (665 nm)",
        "bias": "Rrs_665_bias",
        "rmsd": "Rrs_665_rmsd",
    },
    "adg_412": {
        "name": "Detritus absorption coefficient (412 nm)",
        "bias": "adg_412_bias",
        "rmsd": "adg_412_rmsd",
    },
    "adg_443": {
        "name": "Detritus absorption coefficient (443 nm)",
        "bias": "adg_443_bias",
        "rmsd": "adg_443_rmsd",
    },
    "adg_490": {
        "name": "Detritus absorption coefficient (490 nm)",
        "bias": "adg_490_bias",
        "rmsd": "adg_490_rmsd",
    },
    "adg_510": {
        "name": "Detritus absorption coefficient (510 nm)",
        "bias": "adg_510_bias",
        "rmsd": "adg_510_rmsd",
    },
    "adg_560": {
        "name": "Detritus absorption coefficient (560 nm)",
        "bias": "adg_560_bias",
        "rmsd": "adg_560_rmsd",
    },
    "adg_665": {
        "name": "Detritus absorption coefficient (665 nm)",
        "bias": "adg_665_bias",
        "rmsd": "adg_665_rmsd",
    },
    "aph_412": {
        "name": "Phytoplankton absorption coefficient (412 nm)",
        "bias": "aph_412_bias",
        "rmsd": "aph_412_rmsd",
    },
    "aph_443": {
        "name": "Phytoplankton absorption coefficient (443 nm)",
        "bias": "aph_443_bias",
        "rmsd": "aph_443_rmsd",
    },
    "aph_490": {
        "name": "Phytoplankton absorption coefficient (490 nm)",
        "bias": "aph_490_bias",
        "rmsd": "aph_490_rmsd",
    },
    "aph_510": {
        "name": "Phytoplankton absorption coefficient (510 nm)",
        "bias": "aph_510_bias",
        "rmsd": "aph_510_rmsd",
    },
    "aph_560": {
        "name": "Phytoplankton absorption coefficient (560 nm)",
        "bias": "aph_560_bias",
        "rmsd": "aph_560_rmsd",
    },
    "aph_665": {
        "name": "Phytoplankton absorption coefficient (665 nm)",
        "bias": "aph_665_bias",
        "rmsd": "aph_665_rmsd",
    },
    "kd_490": {
        "name": "Attenuation coefficent (490 nm)",
        "bias": "kd_490_bias",
        "rmsd": "kd_490_rmsd",
    },
    "chlor_a": {
        "name": "Chlorophyll-a concentration",
        "bias": "chlor_a_log10_bias",
        "rmsd": "chlor_a_log10_rmsd",
    },
}

In [ ]:
def xu(ds, v, a):
    x = ds[v][0, 0]
    u = np.sqrt(
        np.square(ds[a["rmsd"]][0, 0]) - np.square(ds[a["bias"]][0, 0])
    )
    if v == "chlor_a":
        u = x * np.sqrt(np.exp(np.square(np.log(10.0) * u)) - 1.0)
    return x, u

In [ ]:
for v, a in config.items():
    x, u = xu(ds, v, a)
    res = (ds[v][1:, 0] - x) / u
    plot_histogram(
        res,
        title=a["name"],
    )
    plot_histogram_mean(
        res,
        dims="i",
        title=a["name"],
    )
    plot_histogram_std(
        res,
        dims="i",
        title=a["name"],
    )
    plot_histogram_range(
        res,
        dims="i",
        title=a["name"],
    )
    plot_mean(
        res,
        dims="i",
        title=a["name"],
    )
    plot_std(
        res,
        dims="i",
        title=a["name"],
    )
    plot_range(
        res,
        dims="i",
        title=a["name"],
    )

In [ ]:
ds.close()

# GHRSST

In [ ]:
ds = xr.open_mfdataset(
    "./200001-ESACCI-L4_GHRSST-SSTdepth-OSTIA-GLOB_CDR3.0-v02.0-fv01.0-1M.*.nc",
    engine="h5netcdf",
    chunks={"lat": 1800, "lon": 3600},
    combine="nested",
    concat_dim="i",
)

In [ ]:
ds

In [ ]:
config = {
    "analysed_sst": {
        "name": "Sea surface temperature",
        "uncertainty": "analysed_sst_uncertainty",
        "distribution": "normal",
    }
}

In [ ]:
def xu(ds, v, a):
    x = ds[v][0, 0]
    u = ds[a["uncertainty"]][0, 0]
    return x, u

In [ ]:
for v, a in config.items():
    x, u = xu(ds, v, a)
    res = (ds[v][1:, 0] - x) / u
    plot_histogram(
        res,
        title=a["name"],
    )
    plot_histogram_mean(
        res,
        dims="i",
        title=a["name"],
    )
    plot_histogram_std(
        res,
        dims="i",
        title=a["name"],
    )
    plot_histogram_range(
        res,
        dims="i",
        title=a["name"],
    )
    plot_mean(
        res,
        dims="i",
        title=a["name"],
    )
    plot_std(
        res,
        dims="i",
        title=a["name"],
    )
    plot_range(
        res,
        dims="i",
        title=a["name"],
    )

In [ ]:
ds.close()

# GLORYS

In [ ]:
ds = xr.open_mfdataset(
    "./mercatorglorys12v1_gl12_mean_200001.*.nc",
    engine="h5netcdf",
    mask_and_scale=True,
    chunks={"latitude": 2041, "lon": 2160, "depth": 1},
    combine="nested",
    concat_dim="i",
)

In [ ]:
ds

In [ ]:
config = {
    "so": {
        "name": "Sea surface salinity",
        "uncertainty": 0.1,
    }
}

In [ ]:
def xu(ds, v, a):
    x = ds[v][0, 0, 0]
    u = a["uncertainty"]
    return x, u

In [ ]:
for v, a in config.items():
    x, u = xu(ds, v, a)
    res = (ds[v][1:, 0, 0] - x) / u
    plot_histogram(
        res,
        title=a["name"],
    )
    plot_histogram_mean(
        res,
        dims="i",
        title=a["name"],
    )
    plot_histogram_std(
        res,
        dims="i",
        title=a["name"],
    )
    plot_histogram_range(
        res,
        dims="i",
        title=a["name"],
    )
    plot_mean(
        res,
        dims=["i"],
        title=a["name"],
    )
    plot_std(
        res,
        dims=["i"],
        title=a["name"],
    )
    plot_range(
        res,
        dims=["i"],
        title=a["name"],
    )

In [ ]:
ds.close()

# ESA SCOPE PP parameters

In [ ]:
ds = xr.open_mfdataset(
    "./PE_mro_mean_error_prv_4km_120p_spring_cor.*.nc",
    engine="h5netcdf",
    chunks={"lat": 1440, "lon": 1440},
    combine="nested",
    concat_dim="i",
)

In [ ]:
ds

In [ ]:
config = {
   "alphaB": {
      "uncertainty": "alphaB_unc",
      "name": "Initial slope of photosynthesis-irradiance curve",
    },
    "PmB": {
      "uncertainty": "PmB_unc",
      "name": "Assimilation number of photosynthesis-irradiance curve",
    }
}

In [ ]:
def xu(ds, v, a):
    x = ds[v][0]
    u = ds[a["uncertainty"]][0]
    return x, u

In [ ]:
for v, a in config.items():
    x, u = xu(ds, v, a)
    res = (ds[v][1:] - x) / u
    plot_histogram(
        res,
        title=a["name"],
    )
    plot_histogram_mean(
        res,
        dims="i",
        title=a["name"],
    )
    plot_histogram_std(
        res,
        dims="i",
        title=a["name"],
    )
    plot_histogram_range(
        res,
        dims="i",
        title=a["name"],
    )
    plot_mean(
        res,
        dims=["i"],
        title=a["name"],
        x="lon",
        y="lat",
    )
    plot_std(
        res,
        dims=["i"],
        title=a["name"],
        x="lon",
        y="lat",
    )
    plot_range(
        res,
        dims=["i"],
        title=a["name"],
        x="lon",
        y="lat",
    )

In [ ]:
ds.close()